In [1]:
from data_loader import data_reader, colnames, colnames_bid
#from inverse_cdf import inverse
from matplotlib import pyplot as plt
import math
import pandas as pd
import numpy as np

In [2]:
imp_19 = data_reader('data\\imp.20131019.txt', colnames, 
                verbose=False, fuck_parsing=True)
bid_19 = data_reader('data\\bid.20131019.txt', colnames_bid, verbose=False, fuck_parsing=True)
# clk_19 = data_reader('data\\clk.20131019.txt', colnames, verbose=False, fuck_parsing=True)
# conv_19 = data_reader('data\\conv.20131019.txt', colnames, verbose=False, fuck_parsing=True)


imp_20 = data_reader('data\\imp.20131020.txt', colnames, 
                verbose=False, fuck_parsing=True)
bid_20 = data_reader('data\\bid.20131020.txt', colnames_bid, verbose=False, fuck_parsing=True)
# clk_20 = data_reader('data\\clk.20131020.txt', colnames, verbose=False, fuck_parsing=True)
# conv_20 = data_reader('data\\conv.20131020.txt', colnames, verbose=False, fuck_parsing=True)

In [16]:
imp = imp_19.append(imp_20, ignore_index=True)
bid = bid_19.append(bid_20, ignore_index=True)
# clk = clk_19.append(clk_20, ignore_index=True)
# conv = conv_19.append(conv_20, ignore_index=True)

In [17]:
imp.shape[0] == imp_19.shape[0] + imp_20.shape[0] and \
bid.shape[0] == bid_19.shape[0] + bid_20.shape[0] # and \
# clk.shape[0] == clk_19.shape[0] + clk_20.shape[0] and \
# conv.shape[0] == conv_19.shape[0] + conv_20.shape[0]

True

In [158]:
bid_rel = bid[['bid_id', 'timestamp']]
imp_rel = imp[['bid_id', 'paying_price']]
# clk_rel = clk[['bid_id', 'paying_price']]
# conv_rel = conv[['bid_id', 'paying_price']]
bids = bid_rel.merge(imp_rel,how='left', on='bid_id').sort_values('timestamp')
bids['not_bought'] = [math.isnan(price) for price in bids.paying_price]

def drop_values_or_fill_zeros(bids, drop=True):
    result = bids.copy()
    if drop:
        print('Dropping not won ads')
        result = bids.dropna()
    else:
        print('Filling not won ads with zeroes')
        result.paying_price = bids.paying_price.fillna(0)
    return result

bids = drop_values_or_fill_zeros(bids, drop=True)

Dropping not won ads


In [159]:
def split_explore_exploit(bids, share_exploration):
    N = bids.shape[0]
    this_many = int(share_exploration * N)
    bids_explore = bids.iloc[:this_many, :]
    bids_exploit = bids.iloc[this_many:, :]
    return bids_explore, bids_exploit, N

share_exploration = 0.001
bids_explore, bids_exploit, N = split_explore_exploit(bids, share_exploration)

G = 1e5
B = 6e6
target_price =  B / G
target_winrate = G / N




print('dimensions: ' + str(bids_explore.shape))
print("target price: %.2f" % target_price)
print("target winrate: %.2f" % target_winrate)

dimensions: (442, 4)
target price: 60.00
target winrate: 0.23


In [160]:
def explore(bids_explore, target_price, N, rounds=5, plot_win_fct=False):
    bid_price = target_price
    N_expl = bids_explore.shape[0]
    step = int(N_expl / rounds)
    start = 0
    stop = step
    spent_exploring = 0
    for round in range(rounds):
        won = 0
        cost = 0
        winning_prices = [np.nan] * step
        index=0
        
        for _, bid in bids_explore \
                            .iloc[start:stop, :] \
                            .iterrows():
            if bid_price > bid.paying_price:
                won += 1
                cost += bid.paying_price
                winning_prices[index] = bid.paying_price
            index += 1
        if plot_win_fct:
            sorted_data = np.sort(winning_prices)
            yvals = np.arange(step) / float(step-1) #len(winning_prices) = step
            plt.plot(sorted_data, yvals)
            plt.show()
        winrate = won / step
        exp_price = cost / won
        exp_views = N * winrate
        spent_exploring += cost
        if  (exp_views < G) or (exp_price < target_price):
            bid_price = bid_price * 2
            print('\nDoubling')
            print("winrate: %.2f" % winrate)
            print("price: %.2f" % exp_price)
        else:
            print('\nBreaking in round %s' % (round + 1))
            print('winrate: %.2f' % winrate)
            print('E(views): %.2f > target_views %.2f' % \
                  (exp_views, G))
            print('E(price): %.2f > target_price %.2f' % \
                  (exp_price, target_price))
            break
        start += step
        stop += step
        
    winning_prices.sort()
    yvalues = np.arange(len(winning_prices)) / float(len(winning_prices)-1)

    cdf = pd.DataFrame({'price' : winning_prices, 
                        'value' : yvalues})
    return cdf, spent_exploring


def exploit(cdf, target_winrate, target_price, N, G):
    
    idx_bG = (np.abs(cdf.value - target_winrate)).argmin()
    b_G = cdf.price[idx_bG]
    
    expected_F = np.array([])
    
    for length in np.arange(1, cdf.shape[0]):
        subset = cdf.price[:length]
        probability = 1 / subset.size
        expected_F = np.append(expected_F, np.nansum(subset) * probability)
        
    idx_bB = (np.abs(expected_F - target_price)).argmin()
    b_B = cdf.price[idx_bB]
    probability_b_B = G / (N * cdf.value[idx_bB])
    
    bid_price = b_B if np.random.uniform(0, 1, 1) < probability_b_B and b_G <= b_B else b_G
    
    return bid_price, b_G, b_B, probability_b_B, idx_bG

In [161]:
cdf, spent_exploring = explore(bids_explore, target_price, N, rounds=10, plot_win_fct=False)



Doubling
winrate: 0.36
price: 27.00

Doubling
winrate: 0.55
price: 58.00

Breaking in round 3
winrate: 1.00
E(views): 442428.00 > target_views 100000.00
E(price): 91.45 > target_price 60.00


In [162]:
spent_exploring

5848.0

In [163]:
param_set = exploit(cdf, target_winrate, target_price, N, G)
print(param_set)

(48.0, 48.0, 122.0, 0.32396985121496225, 10)


In [173]:
# (48.0, 48.0, 122.0, 0.32396985121496225, 10) 
# (0.0, 0.0, 201.0, 0.1539125960230823, 10)
def calculate_winnings(bids_exploit, param_set, B):
    bid_price, b_G, b_B, probability_b_B, idx_bG = param_set
    if b_G < 0.5:
        b_G = 1
    else:
        pass
    index = 0
    cost = 0
    won = 0
    N, _ = bids_exploit.shape
    for _, bid in bids_exploit.iterrows():
        if np.random.uniform(size=1) > probability_b_B:
            bid_price = b_G
        else:
            bid_price = b_B
        if bid_price > bid.paying_price:
            cost += bid.paying_price
            if cost > B:
                break
                print('costs exceeded budget in round %s out of %s' % index, N )
            else:
                won += 1
        index += 1
    win_rate = won / N
    print(win_rate)
    return win_rate, cost
    
    

In [174]:
calculate_winnings(bids_exploit, param_set, B-spent_exploring)

0.3941572809998507


(0.3941572809998507, 5994159.0)

In [176]:
def automatize(bids, share_exploration, B, G):
    bids_explore, bids_exploit, N = split_explore_exploit(bids, share_exploration)
    target_price =  B / G
    target_winrate = G / N
    print('dimensions: ' + str(bids_explore.shape))
    print("target price: %.2f" % target_price)
    print("target winrate: %.2f" % target_winrate)
    print('Exploring')
    cdf, spent_exploring = explore(bids_explore, target_price, N, rounds=10, plot_win_fct=False)
    print('spent exploring: %.2f' % spent_exploring)
    param_set = \
                (bid_price, b_G, b_B, probability_b_B, idx_bG) = \
                exploit(cdf, target_winrate, target_price, N, G)
    print('bid impressions: %.1f' % b_G)
    print('bid budget: %.1f' % b_B)
    print('probability for bid budget: %.1f' % probability_b_B)
    print('Exploiting')
    win_rate, cost = calculate_winnings(bids_exploit, param_set, B - spent_exploring)
    print('win_rate, cost, spent_exploring: %.2f, %.2f, %.2f' % win_rate, cost, spent_exploring)
    return win_rate, cost, spent_exploring

In [183]:
win_rates = []
costs = []
costs_expl = []
for share_exploration in np.linspace(0.0001, 0.1, num=20):
    print('\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('| SHARE of EXPLORATION : %.4f |' % share_exploration)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    win_rate, cost, spent_exploring = automatize(bids, share_exploration, B, G)
    win_rates.append(win_rate)
    costs.append(costs)
    costs_expl.append(spent_exploring)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
| SHARE of EXPLORATION : 0.0001 |
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dimensions: (44, 4)
target price: 60.00
target winrate: 0.23

Doubling
winrate: 0.25
price: 58.00

Breaking in round 2
winrate: 0.50
E(views): 221214.00 > target_views 100000.00
E(price): 91.00 > target_price 60.00
spent exploring: 240.00
bid impressions: nan
bid budget: nan
probability for bid budget: 0.7
0.0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
| SHARE of EXPLORATION : 0.0054 |
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dimensions: (2370, 4)
target price: 60.00
target winrate: 0.23

Doubling
winrate: 0.37
price: 24.76

Doubling
winrate: 0.75
price: 51.89

Breaking in round 3
winrate: 0.98
E(views): 434960.86 > target_views 100000.00
E(price): 78.99 > target_price 60.00
spent exploring: 29744.00
bid impressions: 148.0
bid budget: 122.0
probability for bid budget: 0.3
0.2715119370628417


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
| SHARE of EXPLORATION : 0.0106 |
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dime

KeyboardInterrupt: 